# CLUSTERING - valores de cada película en cada clúster 

Este Notebook asigna un valor a cada película en cada clúster/temática de inmigración (matriz $F2C$), y entrena modelos de regresión para predecir el valor de una película en un clúster. Pasos:

1. [Matriz $F2C$](#f2c)
2. [Modelos de regresión](#modelos)


Inputs:
- Matriz $L2V$, matriz $TFIDF$, clústers finales

Outputs:
- Matriz $F2C$
- Modelos de regresión

In [ ]:
# Importamos librerías 

## Módulos generales
from libraries import *

## Módulos con funciones creadas para este trabajo
## (requieren de haber importado previamente los módulos generales)
from limpieza_subt import *
from clustering import *

In [ ]:
# Completar con directorios 
gitwd = ""
datawd = ""

## Importar datos

In [ ]:
l2v = pd.read_pickle(datawd + "/l2v.pkl")

with open(datawd + "/tfidf.pkl", 'rb') as inputfile: 
    tfidf = pickle.load(inputfile)

filmids = pd.read_pickle(datawd + "/filmids.pkl")

with open(datawd + "/stoi.pkl", 'rb') as inputfile: 
    stoi = pickle.load(inputfile)
    
l2v = l2v.sort_values("stoi").reset_index(drop = True) # Ordenar lemas de L2V según STOI
tfidf = tfidf.toarray()

master = pd.read_csv(datawd + "/titles_master.csv")
msubt = filmids[["tconst", "filmid"]].merge(master,
                                   on = "tconst",
                                   how = "left")

del master

master = pd.read_pickle(datawd + "/master_subt_content_cleaned.pkl")  
master = master[["tconst", "n_tokens"]]
msubt = msubt.merge(master,
                    on = "tconst",
                    how = "left")

del master

msubt = msubt[["tconst", "filmid", "just_migra"]]

msubt.head()

In [ ]:
clusters = pd.read_pickle(datawd + f"/clusters500/final_clusters500.pkl")

<a id='f2c'></a>
## Matriz F2C

Para obtener la matriz película-a-clúster (F2C), multiplicamos dos matrices:

1) tfidf_clusters (F películas x L lemas en los clústers): valor TFIDF para los lemas presentes en los clústers de inmigración
2) clusters_matrix (L lemas en los clústers x C clústers): matrix con lemas en las filas y clústers en la columna, y un 1 indicando que cierto lema pertenece a cierto clúster

F2C = tfidf_clusters . clusters_matrix (F x C)

In [ ]:
# tfidf_clusters
relevant_words = np.unique([item for sublist in clusters.lemmas for item in sublist])
relevant_words = { k:v for k, v in stoi.items() if k in relevant_words}
print("Número de lemas únicos en los clusters:" ,len(relevant_words ))
relevant_words_index = sorted([x for x in relevant_words.values()])
stoi_to_index = dict(zip(relevant_words_index , np.arange(0,len(relevant_words))))
tfidf_clusters = tfidf[:,relevant_words_index] # (F x L) con lemas ordenados por stoi 
print(tfidf_clusters.shape)

# clusters_matrix
aux = clusters.explode("lemmas")
print(aux.shape)
aux["aux"] = 1
aux["stoi"] = aux["lemmas"].map(relevant_words) # agregar STOI para poder combinar con tfidf_clusters
aux = aux.sort_values("stoi").reset_index(drop = True)  
aux["new_index"] = aux.stoi.map(stoi_to_index)
clusters_matrix = sp.coo_matrix( ( aux['aux'], (aux["new_index"], aux['f_cluster_id'])) )
clusters_matrix = clusters_matrix.toarray()
print(clusters_matrix.shape )

# F2C
f2c = np.matmul(tfidf_clusters, clusters_matrix)
print(f2c.shape)
f2c = pd.DataFrame(f2c)
f2c.columns =  clusters["manual_grouping_name"] # películas ya ordenadas por índices
f2c = pd.concat([filmids, f2c], axis = 1)

f2c["clusters_sum"] = f2c.loc[:,clusters["manual_grouping_name"]].sum(axis = 1)

f2c.to_pickle(datawd + f"/clusters{k}/f2c.pkl")
f2c.head()

<a id='modelos'></a>
## Modelos de regresión

### Matriz X
Como matriz de features se promedian los embeddings de todos los lemas de la película

In [ ]:
# Transformar TFIDF a matriz binaria: si una palabra aparece en una peli o no
has_words = tfidf[:,2:] #  quitar columnas de PAD y UNK
has_words[has_words > 0] = 1
del tfidf
print(has_words.shape)
print(l2v.iloc[:,2:-1].shape) # quitando columnas que no son dimensiones del vector de embeddings

# film to vec!
f2v = has_words.dot(l2v.iloc[:,2:-1]) # Suma las dimensiones de todos los lemas de la película
print(f2v.shape)
tot_words = np.sum(has_words, axis = 1) # total de lemas por película
f2v = f2v/ (tot_words[:, np.newaxis]) # dividir por el total de lemas por película (para obtener el promedio)

# pasar a DataFrame y agregar los identificadores de las películas
f2v = pd.DataFrame(f2v)
f2v.columns = ["dim_" + str(x) for x in f2v.columns]
f2v = pd.concat([filmids, f2v], axis = 1)
f2v.to_pickle(datawd + "/X_regre.pkl")
del f2v

dims = [x for x in f2v.columns if "dim_" in x]
X =  f2v[dims] # X es el vector promedio de embeddings para cada película


### Corregir multicolinealidad
Para obtener X_corrected, la matriz de features a usar en los modelos de regresión

In [ ]:
dims = [x for x in f2v.columns if "dim_" in x]
X =  f2v[dims] # X es el vector promedio de embeddings para cada película
multic = abs(X.corr(method='pearson'))

result_df2 =(multic>0.5)  & (multic < 1)  
result_df2 = np.sum(result_df2, axis = 0)  # número de dimensiones con las que correlaciona

print(np.sum(result_df2 > 0)) 
print(np.sum(result_df2 >= np.percentile(result_df2, 95)))

# Quitar variables con alta multicolinealidad
okvars = result_df2.index[result_df2 < np.percentile(result_df2, 95)]
print(len(okvars))
X_corrected = X[okvars]

### Entrenar

In [ ]:
models = ['Benchmark: mean',
          'Benchmark: median',
          'Linear Regression',
          'Lasso Regression',
          'Ridge Regression',
          'Elastic Net Regression',
          'Random Forest Regression']

modelspred = ["mean",
              "median",
              "lr",
              "lasso",
              "ridge",
              "en",
              "rf"]

varname = "manual_grouping_name"

# La matrix Y incluye todas las dependientes a predecir: el valor en cada cluster
Y = f2c[clusters[varname]]


# Entrenamos
train_metrics = pd.DataFrame()
test_metrics = pd.DataFrame()
all_hyperparameters = {}

# entrenamos un modelo para cada clúster como variable dependiente
for c in tqdm(range(len(clusters[varname]))):

    cname = clusters[varname][c]
    y = Y.iloc[:,c]

    seed(9)
    predicted_train, predicted_test, hyperparameters = regression_methods(X_corrected, y) # usar matriz sin multicolinealidad
    
    # guardar valores predichos para cada clúster
    predicted_train.to_pickle(datawd + f"/clusters500/predicted_train_{c}.pkl")
    predicted_test.to_pickle(datawd + f"/clusters500/predicted_test_{c}.pkl")

    # Métricas en TRAIN
    train_m = pd.DataFrame()
    train_m["Algorithms"] = models
    train_m["MSE"] = [ mean_squared_error(predicted_train.y_train, predicted_train   [m]) for m in modelspred]
    train_m["MSE_sd"] = [ mean_squared_error(predicted_train.y_train, predicted_train[m])/np.mean(y**2) for m in modelspred]
    train_m["MAE"] = [ median_absolute_error(predicted_train.y_train, predicted_train[m]) for m in modelspred]
    train_m["c"] = cname

    # Métricas en TEST
    test_m = pd.DataFrame()
    test_m["Algorithms"] = models
    test_m["MSE"] =    [ mean_squared_error   (predicted_test.y_test, predicted_test[m]) for m in modelspred]
    test_m["MSE_sd"] = [ mean_squared_error   (predicted_test.y_test, predicted_test[m])/np.mean(y**2) for m in modelspred]
    test_m["MAE"] =    [ median_absolute_error(predicted_test.y_test, predicted_test[m]) for m in modelspred]                                     
    test_m["c"] = cname  


    train_metrics = pd.concat([train_metrics, train_m], axis = 0)
    test_metrics = pd.concat([test_metrics, test_m], axis = 0)
    all_hyperparameters[cname] = hyperparameters

# Guardar
train_metrics.to_pickle(datawd + f"/clusters500/train_metrics.pkl")
test_metrics.to_pickle(datawd + f"/clusters500/test_metrics.pkl")
with open(datawd + f"/clusters{k}/hyperparameters.pkl", 'wb') as outputfile: 
    pickle.dump(all_hyperparameters, outputfile)


In [ ]:
for x in all_hyperparameters.items():
    print(x)

In [ ]:
# Heatmap con métricas de resultados
train_metrics = pd.read_pickle(datawd + f"/clusters500/train_metrics.pkl")
test_metrics = pd.read_pickle(datawd + f"/clusters500/test_metrics.pkl")

plot_ticks =  ['Benchmark:\n mean', 'Benchmark:\n median', 'Elastic Net \nRegression',
       'Lasso \nRegression', 'Linear\n Regression', 'Random Forest\n Regression',
       'Ridge\n Regression']

# train
df = train_metrics.pivot(index="c", columns="Algorithms", values = "MSE_sd")
fig, ax = plt.subplots(figsize=(10, 5))
df.columns = plot_ticks
ax = sns.heatmap(df, 
                 yticklabels=df.index , 
                 annot=True ,
                 cmap = "gray_r")
ax = plt.xticks(rotation=0)
ax = plt.xlabel('', fontsize=24)
ax = plt.ylabel('', fontsize=24)

ax = plt.title('MSE standardized - Train', fontsize=14)
plt.savefig(datawd + f"/clusters500/metrics_MSEsd_TRAIN.png", dpi = 300, bbox_inches='tight')
ax = plt.show()

# test
df = test_metrics.pivot(index="c", columns="Algorithms", values = "MSE_sd")
df.columns = plot_ticks
fig, ax = plt.subplots(figsize=(10, 5))
ax = sns.heatmap(df, 
                 yticklabels=df.index , 
                 annot=True ,
                 cmap = "gray_r")
ax = plt.xticks(rotation=0)
ax = plt.xlabel('', fontsize=24)
ax = plt.ylabel('', fontsize=24)
ax = plt.title('MSE standardized - Test', fontsize=14)
plt.savefig(datawd + f"/clusters500/metrics_MSEsd_TEST.png", dpi = 300, bbox_inches='tight')
ax = plt.show()
